In [4]:
#pip install gffutils
from collections import defaultdict

import gffutils
import sqlite3

In [5]:
!rm -rf gambiae.g?f.gz ag.db 2>/dev/null
#!wget http://www.vectorbase.org/download/anopheles-gambiae-pestbasefeaturesagamp42gtfgz -O gambiae.gtf.gz
!wget http://www.vectorbase.org/download/anopheles-gambiae-pestbasefeaturesagamp42gff3gz -O gambiae.gff.gz

--2018-05-18 16:43:06--  http://www.vectorbase.org/download/anopheles-gambiae-pestbasefeaturesagamp42gff3gz
Resolving www.vectorbase.org (www.vectorbase.org)... 129.74.255.228
Connecting to www.vectorbase.org (www.vectorbase.org)|129.74.255.228|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.vectorbase.org/download/anopheles-gambiae-pestbasefeaturesagamp42gff3gz [following]
--2018-05-18 16:43:06--  https://www.vectorbase.org/download/anopheles-gambiae-pestbasefeaturesagamp42gff3gz
Connecting to www.vectorbase.org (www.vectorbase.org)|129.74.255.228|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.vectorbase.org/sites/default/files/ftp/downloads/Anopheles-gambiae-PEST_BASEFEATURES_AgamP4.2.gff3.gz [following]
--2018-05-18 16:43:07--  https://www.vectorbase.org/sites/default/files/ftp/downloads/Anopheles-gambiae-PEST_BASEFEATURES_AgamP4.2.gff3.gz
Reusing existing connection to www.vectorbase.org:443.
HTTP

In [6]:
!rm -f ag.db

In [7]:
try:
    db = gffutils.create_db('gambiae.gff.gz', 'ag.db')
except sqlite3.OperationalError:
    db = gffutils.FeatureDB('ag.db')

In [8]:
print(list(db.featuretypes()))
for feat_type in db.featuretypes():
    print(feat_type, db.count_features_of_type(feat_type))

['CDS', 'RNase_P_RNA', 'SRP_RNA', 'contig', 'exon', 'five_prime_UTR', 'gene', 'mRNA', 'miRNA', 'misc_RNA', 'pseudogene', 'rRNA', 'snRNA', 'snoRNA', 'tRNA', 'tRNA_pseudogene', 'three_prime_UTR']
CDS 62408
RNase_P_RNA 1
SRP_RNA 3
contig 8
exon 66485
five_prime_UTR 10520
gene 13624
mRNA 14697
miRNA 187
misc_RNA 10
pseudogene 5
rRNA 53
snRNA 38
snoRNA 12
tRNA 463
tRNA_pseudogene 9
three_prime_UTR 7281


In [9]:
for contig in db.features_of_type('contig'):
    print(contig)

2L	VectorBase	contig	1	49364325	.	.	.	ID=2L;molecule_type=dsDNA;translation_table=1;topology=linear;localization=chromosomal
3R	VectorBase	contig	1	53200684	.	.	.	ID=3R;molecule_type=dsDNA;translation_table=1;topology=linear;localization=chromosomal
UNKN	VectorBase	contig	1	42389979	.	.	.	ID=UNKN;molecule_type=dsDNA;translation_table=1;topology=linear;localization=chromosomal
X	VectorBase	contig	1	24393108	.	.	.	ID=X;molecule_type=dsDNA;translation_table=1;topology=linear;localization=chromosomal
Y_unplaced	VectorBase	contig	1	237045	.	.	.	ID=Y_unplaced;molecule_type=dsDNA;translation_table=1;topology=linear;localization=chromosomal
Mt	VectorBase	contig	1	15363	.	.	.	ID=Mt;molecule_type=dsDNA;translation_table=1;topology=linear;localization=chromosomal
2R	VectorBase	contig	1	61545105	.	.	.	ID=2R;molecule_type=dsDNA;translation_table=1;topology=linear;localization=chromosomal
3L	VectorBase	contig	1	41963435	.	.	.	ID=3L;molecule_type=dsDNA;translation_table=1;topology=linear;localization

In [10]:
num_mRNAs = defaultdict(int)
num_exons = defaultdict(int)
max_exons = 0
max_span = 0
for contig in db.features_of_type('contig'):
    cnt = 0
    for gene in db.region((contig.seqid, contig.start, contig.end), featuretype='gene'):
        cnt += 1
        span = abs(gene.start - gene.end) # strand
        if span > max_span:
            max_span = span
            max_span_gene = gene
        my_mRNAs = list(db.children(gene, featuretype='mRNA'))
        num_mRNAs[len(my_mRNAs)] += 1
        if len(my_mRNAs) == 0:
            exon_check = [gene]
        else:
            exon_check = my_mRNAs
        for check in exon_check:
            num_exons[len(my_exons)] += 1
            if len(my_exons) > max_exons:
                max_exons = len(my_exons)
                max_exons_gene = gene
    print('contig %s, number of genes %d' % (contig.seqid, cnt))
print('Max number of exons: %s (%d)' % (max_exons_gene.id, max_exons))
print('Max span: %s (%d)' % (max_span_gene.id, max_span))
print(num_mRNAs)
print(num_exons)

NameError: name 'my_exons' is not defined